# CNN model

In [ ]:
# Dependencies
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import copy
import sys
import sklearn.metrics as metrics
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError
from torchinfo import summary
from data_formatting import split_sequence_overlap, split_sequence_nooverlap, split_sequence, split_train_test, normalize_data, set_targets
import parameters
import random

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_length):
        super(CNN, self).__init__()
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.kernel_size1 = 7
        self.kernel_size2 = 7
        self.fc_input_size = hidden_size*(seq_length-self.kernel_size1+1-self.kernel_size2+1)
        self.conv1 = nn.Conv2d(1, hidden_size, (self.kernel_size1,1))   # output size : seq_length - 4
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, (self.kernel_size2,1))   # output size : seq_length - 8
        self.fc = nn.Linear(self.fc_input_size, 2)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
    
    def forward(self,x):
        x = torch.unsqueeze(x, 1)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1, self.fc_input_size) #reshaping the data for next dense layer
        x = self.fc(x)
        x[:,1] = self.sig(x[:,1])
        return x

## Parameters

In [ ]:
# Classes we want to predict and binary outputs
list_targets = [0, 3]
list_labels = [0, 1]

# number of subjects used for validation
num_validation_subjects = 1

print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")

In [ ]:
# Transformations 

# Get data
csvfile = sys.argv[1]
train_df = pd.read_csv(csvfile,  delimiter=",")  # 101 features (only AU_r)

# Select only the classes we want to predict
train_df, nclasses, targets_numpy = set_targets(train_df, list_targets, list_labels)

# Convert the subject names (strings) into numbers
subjects = pd.factorize(train_df['Subject'])[0]

# Normalise the features
features_numpy = normalize_data(train_df, parameters.normalise_individual_subjects)
input_dim = features_numpy.shape[1]
print(f"Number of features: {input_dim}")

del train_df

test_accuracies = []
calibrated_test_accuracies = []

# Get distinct subjects
subj = np.unique(subjects)

for test_subj in subj:
  xv_max_val = 0
  avg_test_acc = 0
  val_acc_val_loss_list = []
  test_acc_list = []

  # Cross validation
  for xv in range(parameters.cross_validation_passes):

    test_idx = np.array([test_subj])
    # Take out test subject from trainval (Crooss validation)
    trainval_idx = np.delete(subj, np.where(subj==test_subj))
    val_idx = trainval_idx[random.sample(range(len(trainval_idx)), num_validation_subjects)]
    val_idx = val_idx%len(subj)
    # Remove test & validation subjects from trainval
    train_idx = np.setxor1d(subj, test_idx)
    train_idx = np.setxor1d(train_idx, val_idx)

    print("Generating train/val/test split...")
    features_train, targets_train, features_val, targets_val, features_test, targets_test = split_train_test(targets_numpy, features_numpy, subjects, train_idx, val_idx, test_idx)

    print("Generating sequences...")
    features_train, targets_train = split_sequence_overlap(features_train, targets_train, parameters.seq_dim, parameters.overlap_size)
    features_val, targets_val = split_sequence_overlap(features_val, targets_val, parameters.seq_dim, parameters.overlap_size)
    
    # Overlap or no
    if parameters.test_with_subsequences:
      features_test, targets_test = split_sequence_overlap(features_test, targets_test, parameters.test_seq_dim, parameters.test_overlap_size)
    else:
      features_test, targets_test = split_sequence_nooverlap(features_test, targets_test, parameters.test_seq_dim, parameters.test_overlap_size)

    print(f"Number of training examples: {len(targets_train)}")
    print(f"Number of validation examples: {len(targets_val)}")
    print(f"Number of test examples: {len(targets_test)}")

    # create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
    featuresTrain = torch.from_numpy(features_train)
    targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor)  # data type is long

    featuresVal = torch.from_numpy(features_val)
    targetsVal = torch.from_numpy(targets_val).type(torch.LongTensor)  # data type is long

    # Pytorch train and validation sets
    train = TensorDataset(featuresTrain, targetsTrain)
    val = TensorDataset(featuresVal, targetsVal)
    
    # data loader
    train_loader = DataLoader(train, batch_size=parameters.batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=parameters.batch_size, shuffle=False)

    # create feature and targets tensor for test set.
    if parameters.test_with_subsequences:
      featuresTest = torch.from_numpy(features_test)
      targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor)  # data type is long
      test = TensorDataset(featuresTest, targetsTest)
      test_loader = DataLoader(test, batch_size=parameters.batch_size, shuffle=False)
    
    model = LSTMModel3(input_dim, parameters.hidden_dim, parameters.layer_dim, nclasses, device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2))
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)
    cur_learning_rate = learning_rate
    #scheduler = lrs.ExponentialLR(optimizer, gamma=0.95)

